In [ ]:
#| default_exp Utils.monkey_patching

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from IPython.core.debugger import set_trace

# Monkey Patching

> Modify D3RLPY package behaviour.

In [ ]:
#| export

import numpy as np
import torch
import gym
from betting_env.betting_env import Observation
from inspect import signature
from d3rlpy.torch_utility import _WithDeviceAndScalerProtocol, TorchMiniBatch
from d3rlpy.dataset import TransitionMiniBatch, Transition
from d3rlpy.online.iterators import AlgoProtocol
from d3rlpy.online.explorers import Explorer
from d3rlpy.algos.base import _assert_action_space
from d3rlpy.logger import D3RLPyLogger
from d3rlpy.preprocessing.stack import StackedObservation
from d3rlpy.online.iterators import _setup_algo
from d3rlpy.metrics.scorer import evaluate_on_environment
from d3rlpy.online.buffers import ReplayBuffer
from typing import Any, Callable, Dict, List, Optional, Union
from torch.utils.data._utils.collate import default_collate
from tqdm.auto import trange

## Functions

In this package, we will employ a technique known as `Monkey-patching` which tries to modify and improve the behavior of third-party libraries (in our case, `D3RLPY`).

In this regard, we will modify certain functions supplied in the d3rlpy package in order to adjust our observation output defined in the `betting_env` package.

### torch_api function

This function is included in the d3rlpy `torch_utility.py` file and is designed to send observations,rewards and actions to the appropriate `Scaler` based on their datatype. As a result, we will add our `Observation class` type developed in the `betting_env` package to these criteria.

In [ ]:
# | export


def torch_api(
    scaler_targets: Optional[
        List[str]
    ] = None,  # On which objects the scaler will be applied.
    action_scaler_targets: Optional[
        List[str]
    ] = None,  # On which objects the action scaler will be applied.
    reward_scaler_targets: Optional[
        List[str]
    ] = None,  # On which objects the reward scaler will be applied.
) -> Callable[..., np.ndarray]:
    "Dispatch inputs to provided Scalers (Observations, Actions and Rewards)"

    def _torch_api(f: Callable[..., np.ndarray]) -> Callable[..., np.ndarray]:
        # Get argument names.
        sig = signature(f)
        arg_keys = list(sig.parameters.keys())[1:]

        def wrapper(
            self: _WithDeviceAndScalerProtocol, *args: Any, **kwargs: Any
        ) -> np.ndarray:
            tensors: List[Union[torch.Tensor, TorchMiniBatch]] = []
            # Convert all args to torch.Tensor.
            for i, val in enumerate(args):
                tensor: Union[torch.Tensor, TorchMiniBatch]
                if isinstance(val, torch.Tensor) or isinstance(val, Observation):
                    tensor = val
                elif isinstance(val, list):
                    val = list(self.scaler.transform(v) for v in val)
                    tensor = default_collate(val)
                    tensor = tensor[0].to(self.device).float()

                elif isinstance(val, np.ndarray):
                    if val.dtype == np.uint8:
                        dtype = torch.uint8
                    else:
                        dtype = torch.float32
                    tensor = torch.tensor(
                        data=val.numerical_observation,
                        dtype=dtype,
                        device=self.device,
                    )
                elif val is None:
                    tensor = None
                elif isinstance(val, TransitionMiniBatch):
                    tensor = TorchMiniBatch(
                        val,
                        self.device,
                        scaler=self.scaler,
                        action_scaler=self.action_scaler,
                        reward_scaler=self.reward_scaler,
                    )
                else:
                    tensor = torch.tensor(
                        data=val,
                        dtype=torch.float32,
                        device=self.device,
                    )

                if isinstance(val, Observation):
                    # Preprocess.
                    if self.scaler and scaler_targets:
                        if arg_keys[i] in scaler_targets:
                            tensor = self.scaler.transform(tensor)
                            # Convert to tensor.
                            tensor = torch.tensor(data=tensor, device=self.device)

                    # Preprocess action.
                    if self.action_scaler and action_scaler_targets:
                        if arg_keys[i] in action_scaler_targets:
                            tensor = self.action_scaler.transform(tensor)

                    # Preprocessing reward.
                    if self.reward_scaler and reward_scaler_targets:
                        if arg_keys[i] in reward_scaler_targets:
                            tensor = self.reward_scaler.transform(tensor)

                    # Make sure if the tensor is float32 type.
                    if tensor is not None and tensor.dtype != torch.float32:
                        tensor = tensor.float()

                tensors.append(tensor)
            return f(self, *tensors, **kwargs)

        return wrapper

    return _torch_api

### ReplayBuffer functions

both functions,`append` and `add_last_step` included in the d3rlpy `buffers.py` file, are used to gather `Agent` experiences and interactions with the `Betting environment` in order to train the model. As a result, we will adapt this alteration because our Observation is no longer the same after the transformation step since it has a different shape ( Adding new features).

In [ ]:
#| export


def append(
    self,
    observation: np.ndarray,  # The processed observation, shape = (1, new_shape), new_shape is the number of features extracted from the database in the 'Scaler' call
    action: np.ndarray,  # Chosen action.
    reward: float,  # Reward from the chosen action.
    terminal: float,  # Terminal (done) flag.
    clip_episode: Optional[
        bool
    ] = None,  # Flag to clip the current episode. If ``None``, th episode is clipped based on ``terminal``.
) -> None:
    "Append observation, action, reward and terminal flag to buffer"

    # If None, use terminal.
    if clip_episode is None:
        clip_episode = bool(terminal)

    # Validation.
    if isinstance(action, np.ndarray):
        assert action.shape[0] == self._action_size
    else:
        action = int(action)
        assert action < self._action_size
    # Not allow terminal=True and clip_episode=False
    assert not (terminal and not clip_episode)

    # Create Transition object.
    if self._prev_observation is not None:
        if isinstance(terminal, bool):
            terminal = 1.0 if terminal else 0.0

        transition = Transition(
            observation_shape=(observation.shape[1],),  # Observation shape.
            action_size=self._action_size,  # Number of actions.
            observation=self._prev_observation,  # Current observation.
            action=self._prev_action,  # Chosen action.
            reward=self._prev_reward,  # Profit from the action.
            next_observation=observation,  # Next observation.
            terminal=terminal,  # Done flag.
            prev_transition=self._prev_transition,  # Previous transition.
        )
        if self._prev_transition:
            self._prev_transition.next_transition = transition

        self._transitions.append(transition)
        self._prev_transition = transition

    self._prev_observation = observation
    self._prev_action = action
    self._prev_reward = reward
    self._prev_terminal = terminal

    if clip_episode:
        # skip the timeout state
        if terminal:
            # add the terminal state
            self._add_last_step(observation)
        self.clip_episode()

In [ ]:
#| export


def add_last_step(
    self,
    observation: np.ndarray,  # The last processed observation, shape = (1, new_shape), new_shape is the number of features extracted from the database in the 'Scaler' call
) -> None:
    "Add the observation of the last step."
    # Validation.
    assert self._prev_terminal
    assert self._prev_observation is not None
    # Create Transition object.
    transition = Transition(
        observation_shape=(observation.shape[1],),  # Observation shape.
        action_size=self._action_size,  # Number of actions.
        observation=self._prev_observation,  # Current observation.
        action=self._prev_action,  # Chosen action.
        reward=self._prev_reward,  # Profit from the action.
        next_observation=np.zeros_like(self._prev_observation),  # Next observation.
        terminal=1.0,  # Done flag.
        prev_transition=self._prev_transition,  # Previous transition.
    )
    if self._prev_transition:
        self._prev_transition.next_transition = transition
    self._transitions.append(transition)

## Iterators

This function, which is included in the D3rlpy `iterators.py` file, is intended to start an Online `Deep Reinforcement Learning` training loop. This function takes as input the desired `RL algorithm`, the `environment` in which the `Agent` will learn, a `buffer` in which all agent interactions are recorded, the `explorer`, and the other parameters indicated below.

We used monkey patching in this function to adapt it to our observation datastructure.

In [ ]:
#| export


def train_single_env(
    algo: AlgoProtocol,  # Algorithm object.
    env: gym.Env,  # Gym-like environment.
    buffer: ReplayBuffer,  # Replay buffer.
    explorer: Optional[Explorer] = None,  # Action explorer.
    n_steps: int = 1000000,  # The number of total steps to train.
    n_steps_per_epoch: int = 10000,  # The number of steps per epoch.
    update_interval: int = 1,  # The number of steps per update.
    update_start_step: int = 0,  # The steps before starting updates.
    random_steps: int = 0,  # The steps for the initial random explortion.
    eval_env: Optional[
        gym.Env
    ] = None,  # Gym-like environment. If None, evaluation is skipped.
    eval_epsilon: float = 0.0,  # Epsilon`-greedy factor during evaluation.
    save_metrics: bool = True,  #  Flag to record metrics. If False, the log directory is not created and the model parameters are not saved.
    save_interval: int = 1,  # The number of epochs before saving models.
    experiment_name: Optional[
        str
    ] = None,  # Experiment name for logging. If not passed, the directory name will be ``{class name}_online_{timestamp}``.
    with_timestamp: bool = True,  # Flag to add timestamp string to the last of directory name.
    logdir: str = "d3rlpy_logs",  # Root directory name to save logs.
    verbose: bool = True,  # Flag to show logged information on stdout.
    show_progress: bool = True,  # Flag to show progress bar for iterations.
    tensorboard_dir: Optional[
        str
    ] = None,  # Directory to save logged information in tensorboard (additional to the csv data).  if ``None``, thedirectory will not be created.
    timelimit_aware: bool = True,  # Flag to turn ``terminal`` flag ``False`` when ``TimeLimit.truncated`` flag is ``True``, which is designed to incorporate with ``gym.wrappers.TimeLimit``.
    callback: Optional[
        Callable[[AlgoProtocol, int, int], None]
    ] = None,  # Callable function that takes ``(algo, epoch, total_step)``, which is called at the end of epochs.
) -> None:
    "Start training loop of online deep reinforcement learning."

    # setup logger.
    if experiment_name is None:
        experiment_name = algo.__class__.__name__ + "_online"

    logger = D3RLPyLogger(
        experiment_name,
        save_metrics=save_metrics,
        root_dir=logdir,
        verbose=verbose,
        tensorboard_dir=tensorboard_dir,
        with_timestamp=with_timestamp,
    )
    algo.set_active_logger(logger)

    # Initialize algorithm parameters.
    _setup_algo(algo, env)

    # Test observation shape.(Images or not)
    observation_shape = env.observation_space.shape
    is_image = len(observation_shape) == 3

    # Prepare stacked observation.
    if is_image:
        stacked_frame = StackedObservation(observation_shape, algo.n_frames)

    # Save hyperparameters.
    algo.save_params(logger)

    # Switch based on show_progress flag.
    xrange = trange if show_progress else range

    # Setup evaluation scorer
    eval_scorer: Optional[Callable[..., float]]
    if eval_env:
        eval_scorer = evaluate_on_environment(eval_env, epsilon=eval_epsilon)
    else:
        eval_scorer = None

    # Start training loop.
    observation = env.reset()
    rollout_return = 0.0
    for total_step in xrange(1, n_steps + 1):
        with logger.measure_time("step"):
            # Stack observation if necessary (Image case)
            if is_image:
                stacked_frame.append(observation)
                fed_observation = stacked_frame.eval()
            else:
                observation = observation.astype("f4")
                fed_observation = observation

            # Sample exploration action.
            with logger.measure_time("inference"):
                # Case random exploration.
                if total_step < random_steps:
                    action = env.action_space.sample()
                # Case specified Explorer.
                elif explorer:
                    x = fed_observation.reshape((1,) + fed_observation.shape)
                    action = explorer.sample(algo, x, total_step)[0]
                else:
                    action = algo.sample_action([fed_observation])[0]
            # Step environment.
            with logger.measure_time("environment_step"):
                # Return observation, reward , done and info flags.
                next_observation, reward, terminal, info = env.step(action)
                # Add reward for the current step
                rollout_return += reward

            # Special case for TimeLimit wrapper
            if timelimit_aware and "TimeLimit.truncated" in info:
                clip_episode = True
                terminal = False
            else:
                clip_episode = terminal

            # Apply transformation on observation before buffer storage.
            if algo.scaler:
                observation = algo.scaler.transform(observation)

            # Store observation.
            buffer.append(
                observation=observation,
                action=action,
                reward=reward,
                terminal=terminal,
                clip_episode=clip_episode,
            )
            # Reset if terminated or pass to the next observation.
            if clip_episode:
                observation = env.reset()
                logger.add_metric("rollout_return", rollout_return)
                rollout_return = 0.0
                # for image observation
                if is_image:
                    stacked_frame.clear()
            else:
                observation = next_observation

            # Psuedo epoch count.
            epoch = total_step // n_steps_per_epoch
            if total_step > update_start_step and len(buffer) > algo.batch_size:
                if total_step % update_interval == 0:
                    # Sample mini-batch.
                    with logger.measure_time("sample_batch"):
                        batch = buffer.sample(
                            batch_size=algo.batch_size,
                            n_frames=algo.n_frames,
                            n_steps=algo.n_steps,
                            gamma=algo.gamma,
                        )

                    # Update parameters.
                    with logger.measure_time("algorithm_update"):
                        loss = algo.update(batch)

                    # Record metrics.
                    for name, val in loss.items():
                        logger.add_metric(name, val)

            # Call callback if given.
            if callback:
                callback(algo, epoch, total_step)

        if epoch > 0 and total_step % n_steps_per_epoch == 0:
            # Evaluation.
            if eval_scorer:
                logger.add_metric("evaluation", eval_scorer(algo))

            if epoch % save_interval == 0:
                logger.save_model(total_step, algo)

            # Save metrics.
            logger.commit(epoch, total_step)

    # Clip the last episode.
    buffer.clip_episode()

    # Close logger.
    logger.close()

In [ ]:
#| export


def fit_online(
    self,
    env: gym.Env,  # Gym-like environment.
    buffer: ReplayBuffer,  # Replay buffer.
    explorer: Optional[Explorer] = None,  # Action explorer.
    n_steps: int = 1000000,  # The number of total steps to train.
    n_steps_per_epoch: int = 10000,  # The number of steps per epoch.
    update_interval: int = 1,  # The number of steps per update.
    update_start_step: int = 0,  #  The steps before starting updates.
    random_steps: int = 0,  # The steps for the initial random explortion.
    eval_env: Optional[
        gym.Env
    ] = None,  # Gym-like environment. If None, evaluation is skipped.
    eval_epsilon: float = 0.0,  # Epsilon-greedy factor during evaluation.
    save_metrics: bool = True,  # Flag to record metrics. If False, the log directory is not created and the model parameters are not saved.
    save_interval: int = 1,  #  The number of epochs before saving models.
    experiment_name: Optional[
        str
    ] = None,  # Experiment name for logging. If not passed, the directory name will be ``{class name}_online_{timestamp}``.
    with_timestamp: bool = True,  # Flag to add timestamp string to the last of directory name.
    logdir: str = "d3rlpy_logs",  # Root directory name to save logs.
    verbose: bool = True,  # Flag to show logged information on stdout.
    show_progress: bool = True,  # Flag to show progress bar for iterations.
    tensorboard_dir: Optional[
        str
    ] = None,  # Directory to save logged information in tensorboard (additional to the csv data). If ``None``, the directory will not be created.
    timelimit_aware: bool = True,  # Flag to turn ``terminal`` flag ``False`` when``TimeLimit.truncated`` flag is ``True``, which is designed to incorporate with ``gym.wrappers.TimeLimit``.
    callback: Optional[
        Callable[[AlgoProtocol, int, int], None]
    ] = None,  # Callable function that takes ``(algo, epoch, total_step)``, which is called at the end of epochs.
) -> None:
    "Start training loop of online deep reinforcement learning."

    # Launch training.
    train_single_env(
        algo=self,
        env=env,
        buffer=buffer,
        explorer=explorer,
        n_steps=n_steps,
        n_steps_per_epoch=n_steps_per_epoch,
        update_interval=update_interval,
        update_start_step=update_start_step,
        random_steps=random_steps,
        eval_env=eval_env,
        eval_epsilon=eval_epsilon,
        save_metrics=save_metrics,
        save_interval=save_interval,
        experiment_name=experiment_name,
        with_timestamp=with_timestamp,
        logdir=logdir,
        verbose=verbose,
        show_progress=show_progress,
        tensorboard_dir=tensorboard_dir,
        timelimit_aware=timelimit_aware,
        callback=callback,
    )

In [ ]:
#| hide
from nbdev import nbdev_export

nbdev_export()